# Module 8: 17 - Multi-Agents with Llama-Agents as Microservices
----------------------------------------------------------------
In this lesson, we will explore the use of llama-agents to create and manage multi-agent systems where each agent functions as a microservice. Llama-agents is an async-first framework designed for building, iterating, and productionizing multi-agent systems, which includes multi-agent communication, distributed tool execution, human-in-the-loop processes, and more.

In a llama-agents system, each agent is viewed as a service that continuously processes incoming tasks. Agents interact by pulling and publishing messages from a message queue, enabling seamless communication and coordination.

At the top of the llama-agents system is the control plane. The control plane tracks ongoing tasks, manages the services in the network, and uses an orchestrator to decide which service should handle the next step of a task. This structured approach ensures efficient task management and execution.

## Objectives
* Understand the structure and capabilities of llama-agents for creating multi-agent systems.
* Implement and configure agents as microservices using llama-agents.
* Define tools with hardcoded output for testing and experimentation.
* Observe and analyze the collaboration between agents to solve problems or specific challenges.

## What this session covers:
* Introduction to llama-agents and its role in building multi-agent systems.
* Setting up agents as microservices and enabling their communication via a message queue.
* Defining and testing tools with hardcoded outputs to facilitate agent interaction.
* Utilizing the control plane to manage tasks and orchestrate agent activities.
* Demonstrating how agents collaborate to solve a problem or handle a specific challenge through coordinated task execution.

## Install Libraries

In [ ]:
#!pip install nest_asyncio
#!pip install llama-agents llama-index-agent-openai llama-index-embeddings-openai

## Import Initial Modules

In [1]:
import nest_asyncio
import logging
import os

nest_asyncio.apply()

logging.basicConfig(level=logging.DEBUG)

## Set OpenAI API Key

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

## Define Agents

### SOC Analyst Agent

#### Define Tools

In [ ]:
from llama_index.core.tools import FunctionTool

def get_alert(id: str) -> dict:
    """Returns alert information."""
    alerts = {
        "f96a7f37-e7f1-410f-85d3-93056e6601a5": {
            "alert": "Suspicious Login Attempt Detected",
            "severity": "High",
            "indicators": {
                "failed_login_attempts": 5,
                "successful_login_time": "2024-08-05T10:15:30Z",
                "unfamiliar_ip": "192.0.2.123",
                "geo_location": "Unknown Region",
                "affected_user": "jdoe@example.com"
            }
        },
        "a88c5e2a-2a4b-4848-b6e2-8f726a098b4e": {
            "alert": "Malware Detected in Cloud Storage",
            "severity": "Critical",
            "indicators": {
                "file_name": "invoice2024.exe",
                "file_hash": "d41d8cd98f00b204e9800998ecf8427e",
                "detection_time": "2024-08-05T12:45:00Z",
                "quarantine_status": "Quarantined",
                "malware_family": "RansomwareX",
                "affected_users": ["asmith@example.com", "bjones@example.com"]
            }
        },
        "d1b65937-0d8d-4b7e-8d2f-9e3412e0e1a4": {
            "alert": "Unusual Data Transfer Activity",
            "severity": "Medium",
            "indicators": {
                "data_volume": "150GB",
                "transfer_start_time": "2024-08-04T08:00:00Z",
                "transfer_end_time": "2024-08-04T09:30:00Z",
                "external_ip": "203.0.113.45",
                "source_database": "CustomerDB",
                "initiating_user": "jroe@example.com"
            }
        }
    }
    return alerts.get(id, "Alert ID not found")

get_alert_tool = FunctionTool.from_defaults(fn=get_alert)

In [ ]:
def analyze_user_behavior(user: str) -> dict:
    """Analyzes the behavior of a user account."""
    # Fake behavior analysis
    behavior_analysis = {
        "user": user,
        "login_history": [
            {"time": "2024-08-01T09:00:00Z", "ip": "198.51.100.23"},
            {"time": "2024-08-02T11:00:00Z", "ip": "198.51.100.23"},
            {"time": "2024-08-05T10:15:30Z", "ip": "192.0.2.123"}
        ],
        "anomalous_activities": ["Suspicious Login Attempt"]
    }
    return behavior_analysis

analyze_user_behavior_tool = FunctionTool.from_defaults(fn=analyze_user_behavior)

def analyze_data_transfer(data_volume: str, external_ip: str) -> dict:
    """Analyzes unusual data transfer activities."""
    # Fake data transfer analysis result
    data_transfer_analysis = {
        "data_volume": data_volume,
        "external_ip": external_ip,
        "suspicious": True,
        "potential_exfiltration": True
    }
    return data_transfer_analysis

analyze_data_transfer_tool = FunctionTool.from_defaults(fn=analyze_data_transfer)

#### Define SOC Analyst Agent

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

soc_analyst_tools = [get_alert_tool, analyze_user_behavior_tool, analyze_data_transfer_tool]

soc_analyst_agent = ReActAgent.from_tools(soc_analyst_tools, llm=OpenAI())

### Threat Intelligence Analyst Agent

#### Define Tools

In [ ]:
def analyze_ip(ip: str) -> dict:
    """Enriches information and supports analysis of IP address for reputation and geolocation."""
    # Fake analysis result
    analysis_result = {
        "ip": ip,
        "reputation": "suspicious",
        "geo_location": "Unknown Region",
        "known_associations": ["malware distribution", "botnet activity"]
    }
    return analysis_result

analyze_ip_tool = FunctionTool.from_defaults(fn=analyze_ip)

#### Define TI Agent

In [ ]:
ti_analyst_tools = [analyze_ip_tool]

ti_analyst_agent = ReActAgent.from_tools(ti_analyst_tools, llm=OpenAI())

### Reverse Engineer Agent

#### Define Tools

In [ ]:
def analyze_file(file_hash: str) -> dict:
    """Analyzes a file for malware signatures and behavior."""
    # Fake file analysis result
    file_analysis_result = {
        "file_hash": file_hash,
        "detected_malware": "RansomwareX",
        "behavior": ["encrypts files", "contacts C2 server"]
    }
    return file_analysis_result

analyze_file_tool = FunctionTool.from_defaults(fn=analyze_file)

#### Define Reverse Engineer Agent

In [ ]:
re_tools = [analyze_file_tool]

re_agent = ReActAgent.from_tools(re_tools, llm=OpenAI())

## Define Message Queue

In [ ]:
from llama_agents import SimpleMessageQueue

# create our multi-agent framework components
message_queue = SimpleMessageQueue(port=8000)

## Define Agents as Microservices

In [ ]:
from llama_agents import AgentService

In [ ]:
soc_agent_server = AgentService(
    agent=soc_analyst_agent,
    message_queue=message_queue,
    description="Useful for analyzing alerts.",
    service_name="soc_analyst_agent",
    port=8002,
)
ti_agent_server = AgentService(
    agent=ti_analyst_agent,
    message_queue=message_queue,
    description="Useful for enriching indicators of compromise such as IP Addresses during alert investigations.",
    service_name="ti_analyst_agent",
    port=8003,
)
re_agent_server = AgentService(
    agent=re_agent,
    message_queue=message_queue,
    description="Useful for reversing files during alert investigations.",
    service_name="re_agent",
    port=8004,
)

## Define Orchestrator

In [ ]:
from llama_agents import (
    AgentOrchestrator,
    ControlPlaneServer,
)

control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=OpenAI(model="gpt-4-turbo",)),
    port=8001,
)

## Define Launcher

In [ ]:
from llama_agents import LocalLauncher

# launch it
launcher = LocalLauncher(
    [soc_agent_server, ti_agent_server, re_agent_server],
    control_plane,
    message_queue,
)

## Run Multi-Agent System

In [ ]:
result = launcher.launch_single("Investigate alert f96a7f37-e7f1-410f-85d3-93056e6601a5")

print(f"Result: {result}")